# Backpropagation


# Overarching aims of the exercises

The aim of these exercises is to train the neural network you implemented in the previous exercise.

To train neural networks, we use gradient descent, since there is no analytical expression for the optimal parameters. This means you will need to compute the gradient of the cost function wrt. the network parameters. And then you will need to implement some gradient method.

You will begin by computing gradients for a network with one layer, then two layers, then any number of layers. Keeping track of the shapes and doing things step by step will be very important this week.

We recommend that you do the exercises  by editing and running this notebook file, as it includes some checks along the way that you have implemented the neural network correctly, and running small parts of the code at a time will be important for understanding the methods. If you have trouble running a notebook, you can run this notebook in google colab instead(https://colab.research.google.com/drive/1FfvbN0XlhV-lATRPyGRTtTBnJr3zNuHL#offline=true&sandboxMode=true), though we recommend that you set up VSCode and your python environment to run code like this locally.

First, some setup code that you will need.


In [1]:
import autograd.numpy as np  # We need to use this numpy wrapper to make automatic differentiation work later
from autograd import grad, elementwise_grad
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


# Defining some activation functions
def ReLU(z):
    return np.where(z > 0, z, 0)


# Derivative of the ReLU function
def ReLU_der(z):
    return np.where(z > 0, 1, 0)


def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def mse(predict, target):
    return np.mean((predict - target) ** 2)

# Exercise 1 - Understand the feed forward pass

**a)** Complete last weeks' mandatory exercises if you haven't already.


# Exercise 2 - Gradient with one layer using autograd

For the first few exercises, we will not use batched inputs. Only a single input vector is passed through the layer at a time.

In this exercise you will compute the gradient of a single layer. You only need to change the code in the cells right below an exercise, the rest works out of the box. Feel free to make changes and see how stuff works though!


**a)** If the weights and bias of a layer has shapes (10, 4) and (10), what will the shapes of the gradients of the cost function wrt. these weights and this bias be?

The shape of the gradients of the cost function wrt. weights and bias is the same as the original weights and bias, so (10,4) and (10,)


**b)** Complete the feed_forward_one_layer function. It should use the sigmoid activation function. Also define the weigth and bias with the correct shapes.


In [2]:
def feed_forward_one_layer(W, b, x):
    z = W @ x + b
    a = sigmoid(z)
    return a


def cost_one_layer(W, b, x, target):
    predict = feed_forward_one_layer(W, b, x)
    return mse(predict, target)


x = np.random.rand(2)
target = np.random.rand(3)

W = np.random.randn(3,2)
b = np.random.randn(3)

**c)** Compute the gradient of the cost function wrt. the weigth and bias by running the cell below. You will not need to change anything, just make sure it runs by defining things correctly in the cell above. This code uses the autograd package which uses backprogagation to compute the gradient!


In [3]:
autograd_one_layer = grad(cost_one_layer, [0, 1])
W_g, b_g = autograd_one_layer(W, b, x, target)
print(W_g, b_g)

[[-0.03970906 -0.01412732]
 [-0.03954203 -0.0140679 ]
 [ 0.01948671  0.0069328 ]] [-0.07494318 -0.07462795  0.03677741]


# Exercise 3 - Gradient with one layer writing backpropagation by hand

Before you use the gradient you found using autograd, you will have to find the gradient "manually", to better understand how the backpropagation computation works. To do backpropagation "manually", you will need to write out expressions for many derivatives along the computation.


We want to find the gradient of the cost function wrt. the weight and bias. This is quite hard to do directly, so we instead use the chain rule to combine multiple derivatives which are easier to compute.

$$
\frac{dC}{dW} = \frac{dC}{da}\frac{da}{dz}\frac{dz}{dW}
$$

$$
\frac{dC}{db} = \frac{dC}{da}\frac{da}{dz}\frac{dz}{db}
$$


**a)** Which intermediary results can be reused between the two expressions?

The results for $\frac{dC}{da}\frac{da}{dz}$ can be reused for both expressions, since they are independent of W and b.

**b)** What is the derivative of the cost wrt. the final activation? You can use the autograd calculation to make sure you get the correct result. Remember that we compute the mean in mse.


In [4]:
z = W @ x + b
a = sigmoid(z)

predict = a


def mse_der(predict, target):
    return (2/3)*(predict - target)


print(mse_der(predict, target))

cost_autograd = grad(mse, 0)
print(cost_autograd(predict, target))

[-0.30077475 -0.30041427  0.27117754]
[-0.30077475 -0.30041427  0.27117754]


**c)** What is the expression for the derivative of the sigmoid activation function? You can use the autograd calculation to make sure you get the correct result.


In [5]:
def sigmoid_der(z):
    return np.exp(-z)/(1+np.exp(-z))**2


print(sigmoid_der(z))

sigmoid_autograd = elementwise_grad(sigmoid, 0)
print(sigmoid_autograd(z))

[0.24916713 0.24841679 0.13562113]
[0.24916713 0.24841679 0.13562113]


**d)** Using the two derivatives you just computed, compute this intermetidary gradient you will use later:

$$
\frac{dC}{dz} = \frac{dC}{da}\frac{da}{dz}
$$


In [6]:
dC_da = mse_der(predict,target)
dC_dz = dC_da*sigmoid_der(z)
dC_dz = dC_dz.reshape(-1,1)

**e)** What is the derivative of the intermediary z wrt. the weight and bias? What should the shapes be? The one for the weights is a little tricky, it can be easier to play around in the next exercise first. You can also try computing it with autograd to get a hint.

$$
\frac{dz}{dW} = \frac{\partial}{\partial W} \left(W x + b\right) = x^{T} \quad \quad \text{and} \quad \quad \frac{dz}{db} = 1
$$

The shape of $\frac{dz}{dW}$ should be the trasposed of x, so (1,2). While $\frac{dz}{db} = 1$ is just a scalar. 

**f)** Now combine the expressions you have worked with so far to compute the gradients! Note that you always need to do a feed forward pass while saving the zs and as before you do backpropagation, as they are used in the derivative expressions


In [7]:
x_reshape = x.copy().reshape(-1,1)

dz_dw = x_reshape.T
dz_db = 1

dC_dW = dC_dz @ dz_dw
dC_db = dC_dz*dz_db

dC_db = dC_db.reshape(1,-1)

print(dC_dW, dC_db)

[[-0.03970906 -0.01412732]
 [-0.03954203 -0.0140679 ]
 [ 0.01948671  0.0069328 ]] [[-0.07494318 -0.07462795  0.03677741]]


It appears that we get the same result, as for the autograd solution below.

You should get the same results as with autograd.


In [8]:
W_g, b_g = autograd_one_layer(W, b, x, target)
print(W_g, b_g)

[[-0.03970906 -0.01412732]
 [-0.03954203 -0.0140679 ]
 [ 0.01948671  0.0069328 ]] [-0.07494318 -0.07462795  0.03677741]


# Exercise 4 - Gradient with two layers writing backpropagation by hand


Now that you have implemented backpropagation for one layer, you have found most of the expressions you will need for more layers. Let's move up to two layers.


In [9]:
x = np.random.rand(2).reshape(-1,1)
target = np.random.rand(4).reshape(-1,1)


W1 = np.random.rand(3, 2)
b1 = np.random.rand(3).reshape(-1,1)

W2 = np.random.rand(4, 3)
b2 = np.random.rand(4).reshape(-1,1)

layers = [(W1, b1), (W2, b2)]

In [10]:
z1 = W1 @ x + b1
a1 = sigmoid(z1)
z2 = W2 @ a1 + b2
a2 = sigmoid(z2)
print(a1.shape)

(3, 1)


We begin by computing the gradients of the last layer, as the gradients must be propagated backwards from the end.

**a)** Compute the gradients of the last layer, just like you did the single layer in the previous exercise.


In [11]:

dC_da2 = mse_der(a2,target)
dC_dz2 = dC_da2*sigmoid_der(z2)

dC_dW2 = dC_dz2 @ a1.T
dC_db2 = dC_dz2



To find the derivative of the cost wrt. the activation of the first layer, we need a new expression, the one furthest to the right in the following.

$$
\frac{dC}{da_1} = \frac{dC}{dz_2}\frac{dz_2}{da_1}
$$

**b)** What is the derivative of the second layer intermetiate wrt. the first layer activation? (First recall how you compute $z_2$)

$$
\frac{dz_2}{da_1} = \frac{\partial }{\partial a_1} \left(W_{2}a_1 + b_2\right) = W_{2}^{T}
$$


**c)** Use this expression, together with expressions which are equivelent to ones for the last layer to compute all the derivatives of the first layer.

$$
\frac{dC}{dW_1} = \frac{dC}{da_1}\frac{da_1}{dz_1}\frac{dz_1}{dW_1}
$$

$$
\frac{dC}{db_1} = \frac{dC}{da_1}\frac{da_1}{dz_1}\frac{dz_1}{db_1}
$$


In [12]:
print(dC_dz2.shape)
dC_da1 = W2.T @ dC_dz2 
print(dC_da1.shape)
dC_dz1 = dC_da1*sigmoid_der(z1)
dC_dW1 = dC_dz1 * x.T
dC_db1 = dC_dz1

#print(dC_dW1)

(4, 1)
(3, 1)


In [13]:
print(dC_dW1, dC_db1)
print(dC_dW2, dC_db2)

[[0.00330186 0.00438931]
 [0.00336029 0.00446698]
 [0.0049644  0.0065994 ]] [[0.00529925]
 [0.00539303]
 [0.00796751]]
[[ 0.00564955  0.00440203  0.00488286]
 [-0.00457097 -0.00356162 -0.00395066]
 [ 0.0489398   0.03813301  0.0422983 ]
 [ 0.02176609  0.01695975  0.01881227]] [[ 0.00634069]
 [-0.00513017]
 [ 0.05492692]
 [ 0.02442888]]


**d)** Make sure you got the same gradient as the following code which uses autograd to do backpropagation.


In [14]:
def feed_forward_two_layers(layers, x):
    W1, b1 = layers[0]
    z1 = W1 @ x + b1
    a1 = sigmoid(z1)

    W2, b2 = layers[1]
    z2 = W2 @ a1 + b2
    a2 = sigmoid(z2)

    return a2

In [15]:
def cost_two_layers(layers, x, target):
    predict = feed_forward_two_layers(layers, x)
    return mse(predict, target)


grad_two_layers = grad(cost_two_layers, 0)
grad_two_layers(layers, x, target)

[(array([[0.00247639, 0.00329198],
         [0.00252022, 0.00335024],
         [0.0037233 , 0.00494955]]),
  array([[0.00397444],
         [0.00404477],
         [0.00597563]])),
 (array([[ 0.00423716,  0.00330152,  0.00366215],
         [-0.00342823, -0.00267121, -0.00296299],
         [ 0.03670485,  0.02859975,  0.03172372],
         [ 0.01632457,  0.01271981,  0.0141092 ]]),
  array([[ 0.00475552],
         [-0.00384763],
         [ 0.04119519],
         [ 0.01832166]]))]

**e)** How would you use the gradient from this layer to compute the gradient of an even earlier layer? Would the expressions be any different?

I would use the gradient
$$
\frac{dC}{dz_{l+1}}
$$
and weight $W_{l+1}$ from the earlier layer $l+1$ to compute the gradient for layer $l$.


# Exercise 5 - Gradient with any number of layers writing backpropagation by hand


Well done on getting this far! Now it's time to compute the gradient with any number of layers.

First, some code from the general neural network code from last week. Note that we are still sending in one input vector at a time. We will change it to use batched inputs later.


In [16]:
def create_layers(network_input_size, layer_output_sizes):
    layers = []

    i_size = network_input_size
    for layer_output_size in layer_output_sizes:
        W = np.random.randn(layer_output_size, i_size)
        b = np.random.randn(layer_output_size).reshape(-1,1)
        layers.append((W, b))

        i_size = layer_output_size
    return layers


def feed_forward(input, layers, activation_funcs):
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        z = W @ a + b
        a = activation_func(z)
    return a


def cost(layers, input, activation_funcs, target):
    predict = feed_forward(input, layers, activation_funcs)
    return mse(predict, target)

You might have already have noticed a very important detail in backpropagation: You need the values from the forward pass to compute all the gradients! The feed forward method above is great for efficiency and for using autograd, as it only cares about computing the final output, but now we need to also save the results along the way.

Here is a function which does that for you.


In [17]:
def feed_forward_saver(input, layers, activation_funcs):
    layer_inputs = []
    zs = []
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        layer_inputs.append(a)
        z = W @ a + b
        a = activation_func(z)

        zs.append(z)

    return layer_inputs, zs, a

**a)** Now, complete the backpropagation function so that it returns the gradient of the cost function wrt. all the weigths and biases. Use the autograd calculation below to make sure you get the correct answer.


In [18]:
def backpropagation(
    input, layers, activation_funcs, target, activation_ders, cost_der=mse_der
):
    layer_inputs, zs, predict = feed_forward_saver(input, layers, activation_funcs)

    layer_grads = [() for layer in layers]

    # We loop over the layers, from the last to the first
    for i in reversed(range(len(layers))):
        layer_input, z, activation_der = layer_inputs[i], zs[i], activation_ders[i]

        if i == len(layers) - 1:
            # For last layer we use cost derivative as dC_da(L) can be computed directly
            dC_da = cost_der(predict,target)
        else:
            # For other layers we build on previous z derivative, as dC_da(i) = dC_dz(i+1) * dz(i+1)_da(i)
            (W, b) = layers[i + 1]
            dC_da = W.T @ dC_dz

        dC_dz = dC_da * activation_der(z)
        dC_dW = dC_dz @ layer_input.T
        dC_db = dC_dz

        layer_grads[i] = (dC_dW, dC_db)

    return layer_grads

In [19]:
network_input_size = 2
layer_output_sizes = [3, 4]
activation_funcs = [sigmoid, ReLU]
activation_ders = [sigmoid_der, ReLU_der]

layers = create_layers(network_input_size, layer_output_sizes)

x = np.random.rand(network_input_size).reshape(-1,1)    # creating colloumn vectors
target = np.random.rand(4).reshape(-1,1)                # creating colloumn vectors

In [20]:
layer_grads = backpropagation(x, layers, activation_funcs, target, activation_ders)
print(layer_grads)

[(array([[-0.0087209 , -0.0099403 ],
       [ 0.04099411,  0.04672612],
       [-0.00688114, -0.00784329]]), array([[-0.05036609],
       [ 0.23675462],
       [-0.03974085]])), (array([[0.        , 0.        , 0.        ],
       [0.29415965, 0.21067815, 0.05277896],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]]), array([[-0.        ],
       [ 0.49604922],
       [-0.        ],
       [-0.        ]]))]


In [21]:
cost_grad = grad(cost, 0)
cost_grad(layers, x, [sigmoid, ReLU], target)

[(array([[-0.00654067, -0.00745522],
         [ 0.03074559,  0.03504459],
         [-0.00516085, -0.00588247]]),
  array([[-0.03777457],
         [ 0.17756596],
         [-0.02980563]])),
 (array([[0.        , 0.        , 0.        ],
         [0.22061974, 0.15800861, 0.03958422],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]]),
  array([[0.        ],
         [0.37203691],
         [0.        ],
         [0.        ]]))]

# Exercise 6 - Batched inputs

Make new versions of all the functions in exercise 5 which now take batched inputs instead. See last weeks exercise 5 for details on how to batch inputs to neural networks. You will also need to update the backpropogation function.


In [22]:
def create_layers_batch(network_input_size, layer_output_sizes):
    layers = []

    i_size = network_input_size
    for layer_output_size in layer_output_sizes:
        W = np.transpose(np.random.randn(layer_output_size,i_size))
        b = np.random.randn(layer_output_size)
        layers.append((W, b))

        i_size = layer_output_size
    return layers


def feed_forward_batch(inputs, layers, activation_funcs):
    a = inputs
    for (W, b), activation_func in zip(layers, activation_funcs):
        z = a @ W + b
        a = activation_func(z)
    return a


def cost(layers, input, activation_funcs, target):
    predict = feed_forward(input, layers, activation_funcs)
    return mse(predict, target)

In [23]:
def feed_forward_saver_batch(input, layers, activation_funcs):
    layer_inputs = []
    zs = []
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        layer_inputs.append(a)
        z = a @ W + b
        a = activation_func(z)

        zs.append(z)

    return layer_inputs, zs, a

In [24]:
def backpropagation_batch(
    input, layers, activation_funcs, target, activation_ders, cost_der=mse_der
):
    layer_inputs, zs, predict = feed_forward_saver_batch(input, layers, activation_funcs)

    layer_grads = [() for layer in layers]

    # We loop over the layers, from the last to the first
    for i in reversed(range(len(layers))):
        layer_input, z, activation_der = layer_inputs[i], zs[i], activation_ders[i]

        if i == len(layers) - 1:
            # For last layer we use cost derivative as dC_da(L) can be computed directly
            dC_da = cost_der(predict,target)
        else:
            # For other layers we build on previous z derivative, as dC_da(i) = dC_dz(i+1) * dz(i+1)_da(i)
            (W, b) = layers[i + 1]
            dC_da = W.T @ dC_dz

        dC_dz = dC_da * activation_der(z)
        print(dC_dz.shape, (layer_input.T).shape)
        dC_dW = layer_input.T @ dC_dz 
        dC_db = dC_dz

        layer_grads[i] = (dC_dW, dC_db)

    return layer_grads

In [25]:
network_input_size = 2
data_points = 10
layer_output_sizes = [3, 4]
activation_funcs = [sigmoid, ReLU]
activation_ders = [sigmoid_der, ReLU_der]

layers = create_layers_batch(network_input_size, layer_output_sizes)

x = np.random.rand(data_points,network_input_size)    # creating colloumn vectors
target = np.random.rand(data_points,4)               # creating colloumn vectors

feed_forward_batch(x, layers, activation_funcs)

#layer_grads = backpropagation_batch(x, layers, activation_funcs, target, activation_ders)
#print(layer_grads)

array([[0.        , 0.        , 0.        , 1.06123523],
       [0.        , 0.        , 0.        , 1.18584562],
       [0.        , 0.        , 0.        , 1.14174587],
       [0.        , 0.        , 0.        , 1.15033672],
       [0.        , 0.        , 0.        , 1.09450112],
       [0.        , 0.        , 0.        , 1.06499502],
       [0.        , 0.        , 0.        , 1.10370975],
       [0.        , 0.        , 0.        , 1.06268521],
       [0.        , 0.        , 0.        , 1.0701281 ],
       [0.        , 0.        , 0.        , 1.08133273]])

I didn´t fully manage to create adjust backpropagation for batches, simply because I didnt hav more time.

Therefore I was not able to do exercise 7 unfortunately.

# Exercise 7 - Training


**a)** Complete exercise 6 and 7 from last week, but use your own backpropogation implementation to compute the gradient.

**b)** Use stochastic gradient descent with momentum when you train your network.


# Exercise 8 (Optional) - Object orientation

Passing in the layers, activations functions, activation derivatives and cost derivatives into the functions each time leads to code which is easy to understand in isoloation, but messier when used in a larger context with data splitting, data scaling, gradient methods and so forth. Creating an object which stores these values can lead to code which is much easier to use.

**a)** Write a neural network class. You are free to implement it how you see fit, though we strongly recommend to not save any input or output values as class attributes, nor let the neural network class handle gradient methods internally. Gradient methods should be handled outside, by performing general operations on the layer_grads list using functions or classes separate to the neural network.

We provide here a skeleton structure which should get you started.


In [26]:
class NeuralNetwork:
    def __init__(
        self,
        network_input_size,
        layer_output_sizes,
        activation_funcs,
        activation_ders,
        cost_fun,
        cost_der,
    ):
        pass

    def predict(self, inputs):
        # Simple feed forward pass
        pass

    def cost(self, inputs, targets):
        pass

    def _feed_forward_saver(self, inputs):
        pass

    def compute_gradient(self, inputs, targets):
        pass

    def update_weights(self, layer_grads):
        pass

    # These last two methods are not needed in the project, but they can be nice to have! The first one has a layers parameter so that you can use autograd on it
    def autograd_compliant_predict(self, layers, inputs):
        pass

    def autograd_gradient(self, inputs, targets):
        pass